# Bottleneck analysis

When analyzing the resource estimates of a given quantum program, `bartiq` allows us to identify how these values are being constructed. The recursive, tree-like structure of `bartiq` `Routines` means that finding _algorithmic bottlenecks_ is straightforward. An algorithmic bottleneck refers to a particular subroutine that, perhaps unexpectedly, demands a significant portion of our total resource budget. 

This type of anaylsis allows us to target scientific research onto particularly costly subroutines. By highlighting where our logical resource budget is being spent, we may be able to improve our algorithms further.

Bottleneck analysis is easiest with numeric routines, as comparing symbolic expressions can be tricky. Here, we load in the Double Factorization routine and evaluate it for a given physical system.


In [ ]:
from bartiq import CompiledRoutine, sympy_backend, evaluate
import yaml

with open("../data/double_factorization_compiled.yaml", "r") as f:
    compiled_routine = CompiledRoutine.from_qref(yaml.safe_load(f), sympy_backend)

assignments = {
    "N_spatial": 35,
    "M_r": int(7590 / 298),
    "R": 298,
    "b_as": 10,
    "b_mas": 10,
    "b_givens": 20,
    "lamda": 4,
}
df = evaluate(compiled_routine, assignments=assignments).routine

We can see the exact resource estimates when we call `.resource_values`:

In [173]:
df.resource_values

{'active_volume': 9218713.5,
 'gidney_lelbows': 18380,
 'gidney_relbows': 18380,
 'measurements': 5750,
 'ppms': 0,
 'pprs': 272,
 'rotations': 21,
 't_gates': 217,
 'toffs': 7224}

For this exercise we'll look at the **active volume** resource --- [PsiQuantum's metric](https://arxiv.org/abs/2211.15465) for quantifying the logical cost of a quantum program. Which subroutine(s) are contributing the  most active volume?

The following snippet traverses the `.children` property of `df`, our `CompiledRoutine`, and returns a list of tuples: the name of the subroutine (or `child`) and its individual active volume contribution.

In [183]:
[
    (child_name, av)
    for child_name, child_routine in df.children.items()
    if (av := child_routine.resource_values.get("active_volume", None))
]

[('RotationCatalystStatePrep_compute_0', 23842.5), ('LCU_compute_0', 9194871)]

`LCU_compute_0` ('LCU' referring to the process of [linear combination of unitaries](https://pennylane.ai/qml/demos/tutorial_lcu_blockencoding)) is far and away the dominant contribution of active volume at this level. By running the same snippet on the `LCU` object, we'd get the same information at a layer lower.

Below is a helper function that accepts in a compiled routine object, and an optional resource name, and returns the contributions of the child routines.

In [184]:
def child_contributions_to_parent_resource(
    routine: CompiledRoutine, resource: str = "active_volume"
) -> dict[str, int | float]:
    """For a given routine, check the resource cost of its children."""
    return {
        child_name: resource_value
        for child_name, child_routine in routine.children.items()
        if (resource_value := child_routine.resource_values.get(resource, None))
    }

In [189]:
child_contributions_to_parent_resource(df.children["LCU_compute_0"].children["DoubleFactorizationSelect_compute_0"])

{'DataLookupClean_compute_0': 15506,
 'OneElectronTermBlockEncoding_compute_0': 4573115,
 'z_0': 439,
 'OneElectronTermBlockEncoding_compute_1': 4573164,
 'DataLookupClean_uncompute_0': 4250}

## Exercise

Write a script that recursively finds the most expensive subroutine, and the pathway there. 

## Solutions

In [ ]:
def heaviest_branch(routine: CompiledRoutine, resource: str, pathway: list[str] | None = None):
    pathway = pathway or []
    pathway.append(routine.name)
    if subroutine := routine.children.get(find_most_expensive_subroutine(routine, resource), None):
        pathway = heaviest_branch(subroutine, resource, pathway)
    return pathway


def find_most_expensive_subroutine(routine: CompiledRoutine, resource: str) -> CompiledRoutine | None:
    try:
        return sorted(
            child_contributions_to_parent_resource(routine, resource).items(), key=lambda item: item[1], reverse=True
        )[0][0]
    except IndexError:
        return None